In [2]:
import simpy
import numpy as np
import copy

In [3]:
def example(env):
    print("Before timeout: now=%d"%(env.now))
    value = yield env.timeout(10, value=42)
    print("After timeout: now=%d, value=%d"%(env.now, value))

env = simpy.Environment()
p = env.process(example(env))
env.run()

Before timeout: now=0
After timeout: now=10, value=42


In [4]:
class Example:
    def __init__(self, env, delay=10):
        self.env = env
        self.delay = delay

    def process(self):
        print("Before timeout: now=%d" % (env.now))
        value = yield self.env.timeout(self.delay, value=42)
        print("After timeout: now=%d, value=%d" % (env.now, value))

env = simpy.Environment()
e = Example(env, delay=10)
env.process(e.process())
env.run()

Before timeout: now=0
After timeout: now=10, value=42


In [6]:
class VehicleExample:
    def __init__(self,env: simpy.Environment,min_speed: float = 45.0,max_speed: float = 65,fuel_consumption: float = 23,fuel_capacity: float = 13):
        self.env = env
        self.min_speed = min_speed  # miles/hs
        self.max_speed = max_speed  # miles/hs
        self.fuel_consumption = fuel_consumption  # miles/gal
        self.fuel_capacity = fuel_capacity  # gal
        self.fuel = fuel_capacity  # gal

    def drive(self):
        """Excecutes the loop that makes the vehicle drive"""
        while True:
            print("Start Driving at %.2f" %(self.env.now))
            speed = np.random.randint(self.min_speed, self.max_speed)
            travel_time = (self.fuel * self.fuel_consumption) / (speed)
            yield self.env.timeout(travel_time)
            print("Need refueling at %.2f, was traveling at %.1f mph"%(self.env.now,speed))

            print("Start refueling at %.2f"%(self.env.now))
            yield self.env.timeout(np.random.uniform(0.05, 0.15))
            self.fuel = np.random.randint(self.fuel_capacity - 5, self.fuel_capacity)
            print(
                "Finished refueling at %.2f: fuel now is %.2f\n"%(self.env.now,self.fuel)
            )


env = simpy.Environment()
v = VehicleExample(env)
env.process(v.drive())
env.run(until=24)

Start Driving at 0.00
Need refueling at 5.34, was traveling at 56.0 mph
Start refueling at 5.34
Finished refueling at 5.49: fuel now is 12.00

Start Driving at 5.49
Need refueling at 11.36, was traveling at 47.0 mph
Start refueling at 11.36
Finished refueling at 11.49: fuel now is 8.00

Start Driving at 11.49
Need refueling at 15.10, was traveling at 51.0 mph
Start refueling at 15.10
Finished refueling at 15.23: fuel now is 8.00

Start Driving at 15.23
Need refueling at 18.45, was traveling at 57.0 mph
Start refueling at 18.45
Finished refueling at 18.54: fuel now is 11.00

Start Driving at 18.54
Need refueling at 23.22, was traveling at 54.0 mph
Start refueling at 23.22
Finished refueling at 23.32: fuel now is 11.00

Start Driving at 23.32


In [7]:
class Product:
    def __init__(self, id):
        self.id = id
        self.processed = False

class Machine:
    def __init__(self, env: simpy.Environment, id, process_time=(10, 15, 20)):
        self.env = env
        self.id = id
        self.process_time = process_time
        self.idle = True
    
        self.pending_products = []
        self.processed_products = []
    
    def generate_process_time(self):
        return np.random.triangular(self.process_time[0], self.process_time[1], self.process_time[2])
    
    def process_product(self, product):
        print(f"Processing product {product.id} at time {self.env.now}")
        self.idle = False
        yield self.env.timeout(delay=self.generate_process_time())
        product.processed = True
        self.idle = True
        return product
    
    def _set_production_plan(self, products):
        self.pending_products = copy.deepcopy(products)
        self.processed_products = []

    def process_all_products_randomly(self, products):
        self._set_production_plan(products)
        while len(self.pending_products) > 0:
            next_prod = np.random.choice(range(len(self.pending_products)))
            p = self.pending_products[next_prod]
            yield env.process(self.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)
    
    def process_all_products_fifo(self, products):
        self._set_production_plan(products)
        while len(self.pending_products) > 0:
            p = self.pending_products[0]
            yield env.process(self.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)
    
    def process_all_products_lifo(self, products):
        self._set_production_plan(products)
        while len(self.pending_products) > 0:
            p = self.pending_products[-1]
            yield env.process(self.process_product(p))
            self.pending_products.remove(p)
            self.processed_products.append(p)


In [9]:
print("Processing RANDOM policy")
env = simpy.Environment()
m = Machine(env, id="machine")
products = [Product(id=i) for i in range(5)]
env.process(m.process_all_products_randomly(products))
env.run()

print("Processing FIFO policy")
env = simpy.Environment()
m = Machine(env, id="machine")
products = [Product(id=i) for i in range(5)]
env.process(m.process_all_products_fifo(products))
env.run()

print("Processing LIFO policy")
env = simpy.Environment()
m = Machine(env, id="machine")
products = [Product(id=i) for i in range(5)]
env.process(m.process_all_products_lifo(products))
env.run()

Processing RANDOM policy
Processing product 1 at time 0
Processing product 2 at time 18.357395279372245
Processing product 0 at time 31.76108504842488
Processing product 4 at time 46.05336604489894
Processing product 3 at time 60.20053196690313
Processing FIFO policy
Processing product 0 at time 0
Processing product 1 at time 10.524964529434886
Processing product 2 at time 26.992875286483024
Processing product 3 at time 40.89125875150848
Processing product 4 at time 57.17768850989276
Processing LIFO policy
Processing product 4 at time 0
Processing product 3 at time 12.105677808027606
Processing product 2 at time 27.39431064030356
Processing product 1 at time 44.71589089627303
Processing product 0 at time 58.01643501784348
